# Simple API Example Usage

Author: Ryan Fogle

In [1]:
import os
from cta import SimpleAPI
from dotenv import load_dotenv

load_dotenv(override=True)

True

## GET Time

In [2]:
tracker = SimpleAPI(key=os.environ['CTA_KEY'])
tracker.gettime()

{'bustime-response': {'tm': '20230730 22:55:14'}}

## GET Data Feeds

This is not used for CTA, but included for sake of mimicking CTA HTTP routes. 

In [3]:
tracker.getrtpidatafeeds()

{'bustime-response': {'rtpidatafeeds': [{'name': 'bustime',
    'source': 'Bus Tracker',
    'displayname': 'CTA',
    'enabled': 'true',
    'visible': 'true'}]}}

## GET Routes

In [4]:
tracker.getroutes()

{'bustime-response': {'routes': [{'rt': '1',
    'rtnm': 'Bronzeville/Union Station',
    'rtclr': '#336633',
    'rtdd': '1'},
   {'rt': '2', 'rtnm': 'Hyde Park Express', 'rtclr': '#993366', 'rtdd': '2'},
   {'rt': '3', 'rtnm': 'King Drive', 'rtclr': '#009900', 'rtdd': '3'},
   {'rt': '4', 'rtnm': 'Cottage Grove', 'rtclr': '#cc3300', 'rtdd': '4'},
   {'rt': 'X4',
    'rtnm': 'Cottage Grove Express',
    'rtclr': '#006666',
    'rtdd': 'X4'},
   {'rt': '5',
    'rtnm': 'South Shore Night Bus',
    'rtclr': '#996633',
    'rtdd': '5'},
   {'rt': '6',
    'rtnm': 'Jackson Park Express',
    'rtclr': '#ff0066',
    'rtdd': '6'},
   {'rt': '7', 'rtnm': 'Harrison', 'rtclr': '#666600', 'rtdd': '7'},
   {'rt': '8', 'rtnm': 'Halsted', 'rtclr': '#ff00ff', 'rtdd': '8'},
   {'rt': '8A', 'rtnm': 'South Halsted', 'rtclr': '#66cc66', 'rtdd': '8A'},
   {'rt': '9', 'rtnm': 'Ashland', 'rtclr': '#cc3366', 'rtdd': '9'},
   {'rt': 'X9', 'rtnm': 'Ashland Express', 'rtclr': '#33ff00', 'rtdd': 'X9'},
   {'rt

## GET Directions

In [5]:
tracker.getdirections(rt='8')

{'bustime-response': {'directions': [{'id': 'Northbound',
    'name': 'Northbound'},
   {'id': 'Southbound', 'name': 'Southbound'}]}}

## GET Stops

In [6]:
tracker.getstops(rt='8', dir='Southbound')

{'bustime-response': {'stops': [{'stpid': '5776',
    'stpnm': '1700 N Halsted',
    'lat': 41.912084999999635,
    'lon': -87.6483520000011},
   {'stpid': '5774',
    'stpnm': '1900 N Halsted',
    'lat': 41.916024999998854,
    'lon': -87.64852400000018},
   {'stpid': '5812',
    'stpnm': '2300 S Halsted ',
    'lat': 41.85090499999916,
    'lon': -87.64653299999974},
   {'stpid': '5848',
    'stpnm': '6400 S Halsted',
    'lat': 41.77803900000133,
    'lon': -87.64487099999893},
   {'stpid': '4448',
    'stpnm': 'Archer & Green',
    'lat': 41.84647899999917,
    'lon': -87.64782900000182},
   {'stpid': '5804',
    'stpnm': 'Halsted & 14th Place',
    'lat': 41.86253600000111,
    'lon': -87.6468669999985},
   {'stpid': '5806',
    'stpnm': 'Halsted & 16th Street',
    'lat': 41.85946500000125,
    'lon': -87.64677300000083},
   {'stpid': '17904',
    'stpnm': 'Halsted & 18th Street',
    'lat': 41.857250000001144,
    'lon': -87.64671400000165},
   {'stpid': '17905',
    'stpnm': '

## GET Bustime Predictions

In [7]:
tracker.getpredictions(stpid='5776')

{'bustime-response': {'prd': [{'tmstmp': '20230730 22:55:00',
    'typ': 'A',
    'stpnm': '1700 N Halsted',
    'stpid': '5776',
    'vid': '1200',
    'dstp': 11702,
    'rt': '8',
    'rtdd': '8',
    'rtdir': 'Southbound',
    'des': '79th',
    'prdtm': '20230730 23:07:30',
    'tablockid': '8 -713',
    'tatripid': '1021553',
    'origtatripno': '245961280',
    'dly': False,
    'dyn': 0,
    'prdctdn': '12',
    'zone': '',
    'psgld': '',
    'stst': 82230,
    'stsd': '2023-07-30',
    'flagstop': 0}]}}

## GET Vehicles

In [8]:
tracker.getvehicles(vid='7991')

{'bustime-response': {'vehicle': [{'vid': '7991',
    'tmstmp': '20230730 22:55:03',
    'lat': '41.89920404222276',
    'lon': '-87.64789581298828',
    'hdg': '177',
    'pid': 9368,
    'rt': '8',
    'des': '79th',
    'pdist': 18834,
    'dly': False,
    'tatripid': '1021554',
    'origtatripno': '245961275',
    'tablockid': '8 -702',
    'zone': '',
    'mode': 1,
    'psgld': 'N/A',
    'stst': 81090,
    'stsd': '2023-07-30'}]}}

## GET Patterns (road segments)

In [9]:
tracker.getpatterns(rt='8')

{'bustime-response': {'ptr': [{'pid': 9376,
    'ln': 74690.0,
    'rtdir': 'Northbound',
    'pt': [{'seq': 1,
      'lat': 41.749730000001,
      'lon': -87.64364800000112,
      'typ': 'S',
      'stpid': '3681',
      'stpnm': 'Halsted & 79th Street Terminal',
      'pdist': 0.0},
     {'seq': 2, 'lat': 41.74972, 'lon': -87.644, 'typ': 'W', 'pdist': 0.0},
     {'seq': 3, 'lat': 41.75098, 'lon': -87.64404, 'typ': 'W', 'pdist': 0.0},
     {'seq': 4,
      'lat': 41.75096199999909,
      'lon': -87.64401299999868,
      'typ': 'S',
      'stpid': '17510',
      'stpnm': 'Halsted & 79th Street',
      'pdist': 555.0},
     {'seq': 5, 'lat': 41.75098, 'lon': -87.64404, 'typ': 'W', 'pdist': 0.0},
     {'seq': 6, 'lat': 41.75233, 'lon': -87.64406, 'typ': 'W', 'pdist': 0.0},
     {'seq': 7,
      'lat': 41.75233199999881,
      'lon': -87.64401000000069,
      'typ': 'S',
      'stpid': '5866',
      'stpnm': 'Halsted & 78th Street',
      'pdist': 1048.0},
     {'seq': 8, 'lat': 41.75233,

## GET Service Bulletins  

In [10]:
tracker.getservicebulletins(rt='8')

{'bustime-response': {'sb': [{'nm': '8 Alerts',
    'sbj': 'Check for #8 alerts',
    'dtl': 'For alerts, maps and detailed route information, visit <a href="https://transitchicago.com/bus/8">#8 Halsted Route Information</a> on our website. You can also sign up to have alerts sent by text or e-mail by signing up for <a href="https://transitchicago.com/updates">CTA Updates</a>.',
    'brf': 'For alerts and detailed #8 bus route info, visit https://transitchicago.com/bus/8 - get alerts sent to you via transitchicago.com/updates',
    'prty': 'Low',
    'srvc': [{'rt': '8', 'rtdir': '', 'stpid': '', 'stpnm': ''}],
    'mod': '',
    'url': ''}]}}

## GET Detours

In [11]:
tracker.getdetours(rt='8')

{'bustime-response': {}}

## GET Locale List

In [12]:
tracker.getlocalelist()

{'bustime-response': {'locale': [{'localestring': 'en',
    'displayname': 'English'},
   {'localestring': 'es', 'displayname': 'Spanish'}]}}